In [1]:
# Manipulation de données 
import pandas as pd 
import numpy as np



# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns



# Pour supprimer les messages d'erreurs
import warnings

In [2]:
warnings.filterwarnings("ignore")

# CHARGEMENT DES DONNEES

In [3]:
# Initialisation d'un dataframe vide
df = pd.DataFrame()

# Boucle pour ouvrir et concaténer les 10 fichiers
for i in range(1,11):
    df_to_concat = pd.read_csv(f'Data/avis_ugc_part{i}.csv')
    df = pd.concat([df, df_to_concat])

In [4]:
df.head()

,rating,date,review,likes,user_name,user_nb_reviews,user_nb_photos,cinéma,language
0,1,2024-11-10T07:41:44.880Z,Il n’a pas admissible qu’une grosse chaîne de ...,0,anne degroux,2,0,UGC Ciné Cité Les Halles,fr
1,4,2024-11-09T10:32:32.722Z,"Pratique, plein de salles et bonne programmati...",0,Géraldine Sauvenay,3,0,UGC Ciné Cité Les Halles,fr
2,3,2024-11-09T08:45:17.039Z,"Nettoyez vos écrans, ça commence à se voir !",0,M G,2,0,UGC Ciné Cité Les Halles,fr
3,1,2024-11-08T20:13:38.816Z,The theater still has a bed bug problem. Sat d...,0,Cordelia Ryan,2,1,UGC Ciné Cité Les Halles,fr
4,1,2024-11-08T15:32:14.074Z,VOLUME beaucoup trop fort mon dieu!! On était ...,0,Spei,83,0,UGC Ciné Cité Les Halles,fr


# EXPLORATION DES DONNEES

* Maintenant que toutes nos données sont réunies dans un unique DataFrame, nous allons entamer une phase d'exploration pour mieux comprendre leur contenu. Cela nous permettra de vérifier leur intégrité et de repérer d'éventuels problèmes ou incohérences qui pourraient affecter les étapes d'analyse et de modélisation.

In [5]:
def df_info(df:pd.DataFrame) -> pd.DataFrame:
    """
    Retourne un DataFrame résumant les informations principales sur les colonnes d'un DataFrame donné.
    
    Paramètres:
    df (pd.DataFrame): Le DataFrame à analyser.

    Retourne:
    pd.DataFrame: Un DataFrame contenant le type de données, le nombre de valeurs manquantes
                  et les statistiques descriptives pour chaque colonne.
    """
    
    return pd.concat([df.dtypes, df.isna().sum(), df.describe(include='all').T], axis=1).fillna('')

df_info(df)

,0,1,count,unique,top,freq,mean,std,min,25%,50%,75%,max
rating,int64,0,203186.0,,,,4.040155,1.127952,1.0,4.0,4.0,5.0,5.0
date,object,0,203186.0,109484,2019-04-20T21:10:09.804Z,4,,,,,,,
review,object,115904,87282.0,42339,Bien,626,,,,,,,
likes,int64,0,203186.0,,,,0.160592,1.161993,0.0,0.0,0.0,0.0,53.0
user_name,object,2,203184.0,93439,Thomas,63,,,,,,,
user_nb_reviews,int64,0,203186.0,,,,58.126534,106.120612,0.0,6.0,22.0,65.0,3730.0
user_nb_photos,int64,0,203186.0,,,,144.963674,1272.022506,0.0,0.0,4.0,41.0,200769.0
cinéma,object,0,203186.0,37,UGC Ciné Cité Les Halles,19798,,,,,,,
language,object,0,203186.0,2,fr,110822,,,,,,,


Problèmes initiaux observés :
* Colonne `reviews` : Cette colonne contient de nombreuses valeurs manquantes, ce qui pose un problème, car les avis sont essentiels à notre analyse. Nous devrons donc supprimer les lignes contenant ces valeurs manquantes.
* Colonne `date` : La colonne date n'est pas du bon type de données. Nous devrons la convertir pour pouvoir l'utiliser efficacement dans notre analyse.

In [6]:
# Supprime les valeurs manquantes
df = df.dropna()

# Convertit la colonne 'date' en format datetime, en arrondissant à la minute
df.loc[:, 'date'] = pd.to_datetime(df['date']).dt.floor('min')

In [7]:
df_info(df)

,0,1,count,unique,top,freq,mean,std,min,25%,50%,75%,max
rating,int64,0,87280.0,,,,3.77539,1.322904,1.0,3.0,4.0,5.0,5.0
date,object,0,87280.0,46230,2019-08-12 14:08:00+00:00,6,,,,,,,
review,object,0,87280.0,42338,Bien,626,,,,,,,
likes,int64,0,87280.0,,,,0.369764,1.748912,0.0,0.0,0.0,0.0,53.0
user_name,object,0,87280.0,40977,Thomas,41,,,,,,,
user_nb_reviews,int64,0,87280.0,,,,94.43365,139.576531,0.0,17.0,48.0,116.0,3730.0
user_nb_photos,int64,0,87280.0,,,,214.87964,1688.388514,0.0,0.0,9.0,75.0,200769.0
cinéma,object,0,87280.0,37,UGC Ciné Cité Les Halles,8520,,,,,,,
language,object,0,87280.0,2,fr,47492,,,,,,,


In [8]:
df.duplicated().sum()

560

* Certains avis sont présents en plusieurs exemplaires dans le jeu de données. Nous allons donc supprimer les doublons pour garantir que chaque avis soit unique et éviter les biais dans l'analyse.

In [9]:
# Suppretion des doublons
df = df.drop_duplicates().reset_index(drop=True)

# Vérification
df.duplicated().sum()

0